In [1]:
from visiprog.data import *
from visiprog.metriclearning import *
from visiprog.evaluate import *
%matplotlib inline
import numpy as np
from visiprog.metric_learn.fda_visiprog import LFDA_VISIPROG
from metric_learn import Covariance


from IPython.display import Image, display
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
raw_feature = np.genfromtxt('visiprog/data/curetaliasfree.csv',delimiter=",")
label_material = read_material_label()
groups_VSP, N = read_VSP_label()

print(raw_feature.shape)
print(label_material.shape)

(5245, 82)
(5245,)


In [3]:
label_visiprog = np.zeros((N,))

count = 0
for g in groups_VSP:
    label = False
    
    # check whether any entry in the group has been labelled yet
    # if yes, then merge with that existing label
    for i in g:
        if label_visiprog[i] != 0:
            label = label_visiprog[i]
            break
    
    # if no, then create a new label
    if label == False:
        count += 1
        label = count
        
    for i in g:
        label_visiprog[i] = label

print(np.max(label_visiprog))

202.0


In [4]:
model = Covariance()
model.fit(raw_feature)
X_STSIMM = model.transform(raw_feature)
# print(X_STSIMM)

In [22]:
accuracy_STSIMM = []
accuracy_mat = []
accuracy_vsp_original = []
accuracy_vsp = []

N_material = 61
N_material_train = 30

for fold in range(10):
    try:
        print("Trial {}".format(fold))
        
        # randomly select N_material_train material for training the metric
        # and test on the rest
        train_select = np.zeros_like(label_material).astype(bool)

        idx_train = np.array([])
        for i in range(N_material_train):
            train_class = np.random.randint(0, N_material)
            train_select |= (label_material == train_class)
        
        idx_train = np.where(train_select == True)[0]
        idx_test = np.where(train_select == False)[0]
                
        idx_train = idx_train.astype(int)
        idx_test = idx_test.astype(int)
        
        print("Training shape {} and test shape {}".format(len(idx_train), len(idx_test)))

        X_train = raw_feature[idx_train,:]
        X_STSIMM_train = X_STSIMM[idx_train,:]
        Y_VSP_train = label_visiprog[idx_train]
        Y_mat_train = label_material[idx_train]

        X_test = raw_feature[idx_test,:]
        X_STSIMM_test = X_STSIMM[idx_test,:]
        Y_VSP_test = label_visiprog[idx_test]
        Y_mat_test = label_material[idx_test]

        res_STSIMM = leave_one_sample_out(X_STSIMM_test, Y_mat_test)
        accuracy_STSIMM.append(res_STSIMM['accuracy'])
        print(res_STSIMM['prediction'])
        print(Y_mat_test)
        
        model = LFDA()
        model.fit(X_train, Y_mat_train)
        X_mat_test = model.transform(X_test)
        res = leave_one_sample_out(X_mat_test, Y_mat_test)
        accuracy_mat.append(res['accuracy'])

        model_vsp_original = LFDA()
        model_vsp_original.fit(X_train, Y_VSP_train)
        X_vsp_original_test = model_vsp_original.transform(X_test)
        res_vsp_original = leave_one_sample_out(X_vsp_original_test, Y_mat_test)
        accuracy_vsp_original.append(res_vsp_original['accuracy'])
        
        model_vsp = LFDA_VISIPROG()
        model_vsp.fit(X_train, Y_VSP_train)
        X_vsp_test = model_vsp.transform(X_test)
        res_vsp = leave_one_sample_out(X_vsp_test, Y_mat_test)
        accuracy_vsp.append(res_vsp['accuracy'])

    except Exception as e:
        print(e)

Trial 0
Training shape 2034 and test shape 3211
[15  1  1 ..., 53 53 13]
[ 1  1  1 ..., 53 53 53]
Trial 1
Training shape 2116 and test shape 3129
[17  0  0 ..., 60 60 60]
[ 0  0  0 ..., 60 60 60]
Trial 2
Training shape 2051 and test shape 3194
[ 1  1  1 ..., 52 57 57]
[ 1  1  1 ..., 57 57 57]
Trial 3
Training shape 1904 and test shape 3341
[21  0  0 ..., 60 60 60]
[ 0  0  0 ..., 60 60 60]
Trial 4
Training shape 2087 and test shape 3158
[18  0  0 ..., 55 55 55]
[ 0  0  0 ..., 55 55 55]
Trial 5
Training shape 2419 and test shape 2826
[48  3  3 ..., 59 59 58]
[ 3  3  3 ..., 59 59 59]
Trial 6
Training shape 2049 and test shape 3196
[15  1  1 ..., 60 60 60]
[ 1  1  1 ..., 60 60 60]
Trial 7
Training shape 2367 and test shape 2878
[21  0  0 ..., 60 60 60]
[ 0  0  0 ..., 60 60 60]
Trial 8
Training shape 1784 and test shape 3461
[51  2  2 ..., 60 60 60]
[ 2  2  2 ..., 60 60 60]
Trial 9
Training shape 2252 and test shape 2993
[21  0  0 ..., 60 60 60]
[ 0  0  0 ..., 60 60 60]


In [25]:
print(len(accuracy_mat))
print(len(accuracy_vsp))
print(len(accuracy_STSIMM))
print(len(accuracy_vsp_original))

print(res_STSIMM['accuracy'])
print(accuracy_STSIMM)

10
10
10
10
0.890410958904
[0.90501401432575523, 0.8983700862895494, 0.87257357545397618, 0.92427416941035623, 0.89328689043698539, 0.92108987968860578, 0.88892365456821021, 0.89263377345378736, 0.89685062120774339, 0.8904109589041096]


In [26]:
df = pd.DataFrame({'visiprog':accuracy_vsp, 'material':accuracy_mat, 'STSIMM':accuracy_STSIMM, \
       'visiprog_original':accuracy_vsp_original}, columns=['STSIMM','material','visiprog_original','visiprog'])
display(df)
display(df.mean())

,STSIMM,material,visiprog_original,visiprog
0,0.905014,0.987543,0.976954,0.988789
1,0.898370,0.989134,0.978268,0.989134
2,0.872574,0.989042,0.983719,0.990294
3,0.924274,0.985334,0.981143,0.988028
4,0.893287,0.989867,0.962951,0.985434
5,0.921090,0.992569,0.984430,0.991154
6,0.888924,0.992178,0.978723,0.992804
7,0.892634,0.987144,0.974635,0.987839
8,0.896851,0.989887,0.987865,0.989021
9,0.890411,0.989977,0.961243,0.989977


STSIMM               0.898343
material             0.989267
visiprog_original    0.976993
visiprog             0.989247
dtype: float64

In [ ]:
print(model.transformer())